### Sex Ratio By Last Name (Part I: Preprocessing in rolls dataset)

Indian names are famously complicated. To infer last names, we create rules that lead us to make fewer false positives than negatives.

Here's the algorithm we use:

1. Start with `orig_df`. Create a column called `birth_year` (year - age).

2. There are multiple steps to filtering to the base data frame:

    * Split names by space and store each word in a separate column. Assume the last word to be the last name and store in column `last_name.`

    * Filter out one-word names and where the last word is fewer than two characters. We lose XXX records as a result of that.

    * Filter out names with non-alphabetical characters. We lose XXX records as a result of that.

    * Remove records of people who are recorded as being born before 1900 as we think those birth dates are unreliable. We lose XXX rows.

    * Remove people of "Third Gender" because we only have 8 rows of people recorded as third gender.

    * Store the data frame as `base_df`.

In [1]:
import pandas as pd
from glob import glob
from functools import partial
import gc


In [2]:
fns = sorted(glob('../age_gap/*.csv.gz'))
orig_df = pd.concat(map(partial(pd.read_csv, usecols=['elector_name', 'house_no', 'age', 'sex', 'state', 'year'], dtype=str), fns), ignore_index=True)
orig_df

,elector_name,house_no,age,sex,year,state
0,PINTU DATTA,-,38,Male,2017,andaman
1,SUSHILA DEVI,-,39,Female,2017,andaman
2,BALA KRISHNA,-,29,Male,2017,andaman
3,CHANDRA SEKHAR,-,45,Male,2017,andaman
4,SIKANDER,-,49,Male,2017,andaman
...,...,...,...,...,...,...
598064089,vivek chandr chouhaan,614.0,34,Male,2018,uttarakhand
598064090,renoo chouhaan,614.0,31,Female,2018,uttarakhand
598064091,abhinav kumaar,614.0,31,Male,2018,uttarakhand
598064092,priyamkaa raanee,614.0,30,Female,2018,uttarakhand


In [3]:
# Take out null and invalid age
orig_df.dropna(subset=['age'], inplace=True)
orig_df.drop(orig_df[orig_df.age.str.len() > 3].index, inplace=True)
orig_df.age = orig_df.age.astype(int)
orig_df.year = orig_df.year.astype(int)
orig_df['birth_year'] = orig_df.year - orig_df.age
del orig_df['year']
gc.collect()
orig_df

,elector_name,house_no,age,sex,state,birth_year
0,PINTU DATTA,-,38,Male,andaman,1979
1,SUSHILA DEVI,-,39,Female,andaman,1978
2,BALA KRISHNA,-,29,Male,andaman,1988
3,CHANDRA SEKHAR,-,45,Male,andaman,1972
4,SIKANDER,-,49,Male,andaman,1968
...,...,...,...,...,...,...
598064089,vivek chandr chouhaan,614.0,34,Male,uttarakhand,1984
598064090,renoo chouhaan,614.0,31,Female,uttarakhand,1987
598064091,abhinav kumaar,614.0,31,Male,uttarakhand,1987
598064092,priyamkaa raanee,614.0,30,Female,uttarakhand,1988


In [4]:
# Take out null elector_name and change to upper case
orig_df.dropna(subset=['elector_name'], inplace=True)
orig_df.elector_name = orig_df.elector_name.str.lower()
orig_df['name_wc'] = orig_df.elector_name.apply(lambda c: len(c.split()))
orig_df

,elector_name,house_no,age,sex,state,birth_year,name_wc
0,pintu datta,-,38,Male,andaman,1979,2
1,sushila devi,-,39,Female,andaman,1978,2
2,bala krishna,-,29,Male,andaman,1988,2
3,chandra sekhar,-,45,Male,andaman,1972,2
4,sikander,-,49,Male,andaman,1968,1
...,...,...,...,...,...,...,...
598064089,vivek chandr chouhaan,614.0,34,Male,uttarakhand,1984,3
598064090,renoo chouhaan,614.0,31,Female,uttarakhand,1987,2
598064091,abhinav kumaar,614.0,31,Male,uttarakhand,1987,2
598064092,priyamkaa raanee,614.0,30,Female,uttarakhand,1988,2


In [5]:
orig_df['last_name'] = None
orig_df.loc[orig_df.name_wc >= 2, 'last_name'] = orig_df.elector_name.apply(lambda c: c.split()[-1])
orig_df.loc[orig_df.last_name.notnull() & (orig_df.last_name.str.len() < 2), 'last_name'] = None
orig_df.dropna(subset=['last_name'], inplace=True)
del orig_df['name_wc']
gc.collect()
orig_df

,elector_name,house_no,age,sex,state,birth_year,last_name
0,pintu datta,-,38,Male,andaman,1979,datta
1,sushila devi,-,39,Female,andaman,1978,devi
2,bala krishna,-,29,Male,andaman,1988,krishna
3,chandra sekhar,-,45,Male,andaman,1972,sekhar
7,marianus toppo,-,47,Male,andaman,1970,toppo
...,...,...,...,...,...,...,...
598064089,vivek chandr chouhaan,614.0,34,Male,uttarakhand,1984,chouhaan
598064090,renoo chouhaan,614.0,31,Female,uttarakhand,1987,chouhaan
598064091,abhinav kumaar,614.0,31,Male,uttarakhand,1987,kumaar
598064092,priyamkaa raanee,614.0,30,Female,uttarakhand,1988,raanee


In [6]:
# Drop a bunch of non-alpha last_name
orig_df.drop(orig_df[~orig_df.last_name.str.isalpha()].index, inplace=True)
orig_df

,elector_name,house_no,age,sex,state,birth_year,last_name
0,pintu datta,-,38,Male,andaman,1979,datta
1,sushila devi,-,39,Female,andaman,1978,devi
2,bala krishna,-,29,Male,andaman,1988,krishna
3,chandra sekhar,-,45,Male,andaman,1972,sekhar
7,marianus toppo,-,47,Male,andaman,1970,toppo
...,...,...,...,...,...,...,...
598064089,vivek chandr chouhaan,614.0,34,Male,uttarakhand,1984,chouhaan
598064090,renoo chouhaan,614.0,31,Female,uttarakhand,1987,chouhaan
598064091,abhinav kumaar,614.0,31,Male,uttarakhand,1987,kumaar
598064092,priyamkaa raanee,614.0,30,Female,uttarakhand,1988,raanee


In [7]:
# drop birth year before 1900
orig_df.drop(orig_df[orig_df.birth_year < 1900].index, inplace=True)
orig_df

,elector_name,house_no,age,sex,state,birth_year,last_name
0,pintu datta,-,38,Male,andaman,1979,datta
1,sushila devi,-,39,Female,andaman,1978,devi
2,bala krishna,-,29,Male,andaman,1988,krishna
3,chandra sekhar,-,45,Male,andaman,1972,sekhar
7,marianus toppo,-,47,Male,andaman,1970,toppo
...,...,...,...,...,...,...,...
598064089,vivek chandr chouhaan,614.0,34,Male,uttarakhand,1984,chouhaan
598064090,renoo chouhaan,614.0,31,Female,uttarakhand,1987,chouhaan
598064091,abhinav kumaar,614.0,31,Male,uttarakhand,1987,kumaar
598064092,priyamkaa raanee,614.0,30,Female,uttarakhand,1988,raanee


In [8]:
orig_df['sex'] = orig_df.sex.str.lower()
orig_df.drop(orig_df[orig_df.sex=='third gender'].index, inplace=True)
orig_df

,elector_name,house_no,age,sex,state,birth_year,last_name
0,pintu datta,-,38,male,andaman,1979,datta
1,sushila devi,-,39,female,andaman,1978,devi
2,bala krishna,-,29,male,andaman,1988,krishna
3,chandra sekhar,-,45,male,andaman,1972,sekhar
7,marianus toppo,-,47,male,andaman,1970,toppo
...,...,...,...,...,...,...,...
598064089,vivek chandr chouhaan,614.0,34,male,uttarakhand,1984,chouhaan
598064090,renoo chouhaan,614.0,31,female,uttarakhand,1987,chouhaan
598064091,abhinav kumaar,614.0,31,male,uttarakhand,1987,kumaar
598064092,priyamkaa raanee,614.0,30,female,uttarakhand,1988,raanee


In [9]:
orig_df.to_csv('in_rolls_last_sex.tar.gz', index=False, compression="gzip")


In [ ]:
# Resource List: procs=1,mem=200gb
# Resources Used: cput=01:46:17,vmem=189763324kb,walltime=01:46:25,mem=198019236kb,energy_used=0